In [2]:
import pandas as pd

In [3]:
dataset_url = "https://raw.githubusercontent.com/nick-kann/Xatu-AI/main/dataset.csv"
df = pd.read_csv(dataset_url)

In [4]:
df = df.sample(n=50000)
df

,id,p1_poke1,p1_poke2,p1_poke3,p1_poke4,p1_poke5,p1_poke6,p2_poke1,p2_poke2,p2_poke3,p2_poke4,p2_poke5,p2_poke6,p1_choice,p2_choice
49245,49246,Ogerpon-Wellspring,Cinderace,Landorus-Therian,Dondozo,Slowking-Galar,Tyranitar,Ogerpon-Wellspring,Mamoswine,Thundurus-Therian,Donphan,Samurott-Hisui,Darkrai,Landorus-Therian,Darkrai
7739,7740,Iron Treads,Ribombee,Kyurem,Iron Valiant,Samurott-Hisui,Serperior,Dragapult,Kingambit,Glimmora,Zamazenta,Primarina,Roaring Moon,Ribombee,Primarina
2986,2987,Cinderace,Rillaboom,Ting-Lu,Dondozo,Raging Bolt,Corviknight,Sinistcha,Cobalion,Maushold,Alomomola,Samurott-Hisui,Gliscor,Raging Bolt,Gliscor
60058,60059,Ogerpon-Wellspring,Meowscarada,Landorus-Therian,Skarmory,Garganacl,Volcarona,Clefable,Cinderace,Landorus-Therian,Dondozo,Zamazenta,Muk-Alola,Landorus-Therian,Clefable
31429,31430,Rillaboom,Great Tusk,Ogerpon-Cornerstone,Hawlucha,Heatran,Hatterene,Dragapult,Iron Moth,Iron Treads,Greninja-*,Dragonite,Serperior,Hatterene,Iron Treads
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12843,12844,Ogerpon-Wellspring,Dragapult,Great Tusk,Garganacl,Raging Bolt,Tornadus-Therian,Iron Moth,Landorus-Therian,Gouging Fire,Kingambit,Primarina,Roaring Moon,Ogerpon-Wellspring,Roaring Moon
29560,29561,Ogerpon-Wellspring,Cresselia,Kingambit,Ursaluna,Hatterene,Slowking-Galar,Landorus-Therian,Barraskewda,Hydrapple,Samurott-Hisui,Pelipper,Gholdengo,Hatterene,Samurott-Hisui
2840,2841,Excadrill,Kommo-o,Garganacl,Glimmora,Dragonite,Cramorant,Cinderace,Landorus-Therian,Kyurem,Kingambit,Iron Valiant,Slowking-Galar,Garganacl,Landorus-Therian
60063,60064,Dragapult,Great Tusk,Keldeo-Resolute,Kingambit,Garganacl,Slowking-Galar,Volcanion,Weezing,Dondozo,Moltres,Blissey,Gliscor,Garganacl,Gliscor


In [5]:
df.dropna(inplace=True)
print(df.isnull().sum())

id           0
p1_poke1     0
p1_poke2     0
p1_poke3     0
p1_poke4     0
p1_poke5     0
p1_poke6     0
p2_poke1     0
p2_poke2     0
p2_poke3     0
p2_poke4     0
p2_poke5     0
p2_poke6     0
p1_choice    0
p2_choice    0
dtype: int64


In [6]:
X = df.drop(['p1_choice', 'p2_choice', 'id'], axis=1)
y = df['p2_choice']
categories = X.columns.to_list()
# X = pd.get_dummies(X, columns=categories)

In [12]:
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X)

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

lgb = LGBMClassifier(objective='multiclass', verbose=-1)
lgb.fit(X_train, y_train)

LGBMClassifier(objective='multiclass', verbose=-1)

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_pred = model.predict(X_test)
predicted_names = le.inverse_transform(predictions)

probabilities = model.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.023


In [83]:
from sklearn.neighbors import KNeighborsClassifier
from collections import defaultdict

k_values = range(1, 500)
knn_accuracy_dict = defaultdict(float)

X_encoded = encoder.fit_transform(X)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

for k in k_values:
    print(f'k: {k}/500 completed', end='\r')
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    knn_accuracy_dict[k] = accuracy
    

In [91]:
best_k = sorted(knn_accuracy_dict, key=knn_accuracy_dict.get, reverse=True)[:3]

knn_top_acc = {}
for key in best_k:
    knn_top_acc[key] = round(knn_accuracy_dict[key], 3)

print(knn_top_acc)

{32: 0.384, 33: 0.384, 31: 0.383}


In [96]:
from sklearn.naive_bayes import GaussianNB

X_encoded = encoder.fit_transform(X).toarray()
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
gnb_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {gnb_accuracy:.3f}")

Accuracy: 0.099


In [97]:
from sklearn.naive_bayes import MultinomialNB

X_encoded = encoder.fit_transform(X).toarray()
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

mnb = GaussianNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)
mnb_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {mnb_accuracy:.3f}")

Accuracy: 0.099


In [98]:
from sklearn.ensemble import RandomForestClassifier

X_encoded = encoder.fit_transform(X).toarray()
le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

forest100 = RandomForestClassifier(n_estimators=100)
forest100.fit(X_train, y_train)
y_pred = forest100.predict(X_test)
forest100_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {forest100_accuracy:.3f}")

Accuracy: 0.474
